In [1]:
%%capture
import os
import site
site.addsitedir('/home/schirrmr/.local/lib/python2.7/site-packages/')
site.addsitedir('/usr/lib/pymodules/python2.7/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
%cd /home/schirrmr/braindecode/code/braindecode/
assert 'THEANO_FLAGS' in os.environ
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'

In [2]:
%load_ext autoreload
%autoreload 2
import matplotlib
from matplotlib import pyplot
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 7

import matplotlib.lines as mlines
import seaborn
seaborn.set_style('darkgrid')

In [3]:
import theano

In [4]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


## Load Twice current way, see if equality test works

In [214]:
from braindecode.datasets.bbci_dataset import ProcessedDataset, BCICompetition4Set2A
import numpy as np
from copy import deepcopy
filename='data/bci-competition-iv/2a-combined/A03TE.mat'
wyrm_set = BCICompetition4Set2A(filename=filename)
cnt = wyrm_set.load()

In [141]:
filename='data/bci-competition-iv/2a-combined/A03TE.mat'
wyrm_set2 = BCICompetition4Set2A(filename=filename)
cnt2 = wyrm_set2.load()

In [142]:
cnt == cnt2

True

In [165]:
cnt.axes[1]

array(['EEG-Fz', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG-C3', 'EEG',
       'EEG-Cz', 'EEG', 'EEG-C4', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG',
       'EEG', 'EEG-Pz', 'EEG', 'EEG', 'EOG-left', 'EOG-central',
       'EOG-right'], 
      dtype='|S11')

In [231]:
from braindecode.datasets.bbci_dataset import ProcessedDataset, BBCIDataset
wyrm_set3 = BBCIDataset(filename='data/bci-competition-iv/2a-combined-bbci/A03TE.bbci.mat',
                       load_sensor_names=[c for c in chan_names if c.startswith('EEG')] )
#wyrm_set3 = BBCIDataset(filename='data/bci-competition-iv/2a-combined-bbci/A03TE.bbci.mat',
#                       load_sensor_names=[c for c in chan_names] )
cnt3 = wyrm_set3.load()



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


In [233]:
np.any(np.isnan(cnt3.data))

False

In [229]:
cnt3.data[np.isnan(cnt3.data)] = np.nanmean(cnt3.data, axis=0)

ValueError: NumPy boolean array indexing assignment cannot assign 22 input values to the 0 output values where the mask is true

In [185]:
cnt3.data.shape

(1309305, 25)

In [186]:
cnt.data.shape

(1309305, 25)

In [217]:
np.allclose(cnt.data, cnt3.data)

False

In [210]:
np.array_equal(cnt3.axes[0], cnt.axes[0])

True

In [211]:
np.array_equal(cnt.markers, cnt3.markers)

True

In [212]:
cnt.fs == cnt3.fs

True

In [213]:
cnt3 == cnt

False

In [52]:
h5file_old = h5py.File('data/BBCI-without-last-runs/AnWeMoSc1S001R01_ds10_1-12.BBCI.mat', 'r')

h5file_old['mrk']['time']

<HDF5 dataset "time": shape (935, 1), type "<f8">

In [17]:
h5file_old['mrk']['event']['desc']

<HDF5 dataset "desc": shape (1, 935), type "<u2">

In [18]:
h5file_old['mrk']['event']['desc'].dtype

dtype('uint16')

## real stuffs

In [166]:
samples = cnt.data.shape[0]

In [167]:
chan_names = cnt.axes[1]

In [168]:
for i_chan in xrange(len(chan_names)):
    if chan_names[i_chan] == 'EEG':
        chan_names[i_chan] = 'EEG-' + str(i_chan)

In [169]:
chan_names

array(['EEG-Fz', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 'EEG-6',
       'EEG-C3', 'EEG-8', 'EEG-Cz', 'EEG-10', 'EEG-C4', 'EEG-12', 'EEG-13',
       'EEG-14', 'EEG-15', 'EEG-16', 'EEG-17', 'EEG-18', 'EEG-Pz',
       'EEG-20', 'EEG-21', 'EOG-left', 'EOG-central', 'EOG-right'], 
      dtype='|S11')

In [170]:
sampling_rate = float(cnt.fs)

In [171]:
signal = cnt.data

In [172]:
event_times_in_ms, event_classes = [np.array(tupl, dtype=np.float64) for tupl in zip(*cnt.markers)]

## Writing

In [173]:
rm data/bci-competition-iv/2a-combined-bbci/A03TE.bbci.mat

In [174]:
import h5py
h5file = h5py.File('data/bci-competition-iv/2a-combined-bbci/A03TE.bbci.mat', 'w')

In [175]:
dat_group = h5file.create_group('dat')
nfo_group = h5file.create_group('nfo')

In [176]:
dat_group.create_dataset("fs", data=np.array(sampling_rate).astype(np.float64), shape=[1,1])

<HDF5 dataset "fs": shape (1, 1), type "<f8">

In [177]:
nfo_group.create_dataset('T', data=samples, dtype=np.int64, shape=(1,1))

<HDF5 dataset "T": shape (1, 1), type "<i8">

In [178]:
dat_group.create_dataset('clab', data=chan_names, shape=[chan_names.shape[0], 1])

<HDF5 dataset "clab": shape (25, 1), type "|S11">

In [179]:
mrk_group = h5file.create_group('mrk')
event_group = mrk_group.create_group('event')

In [180]:
event_group.create_dataset('desc', data=event_classes, dtype=np.uint16, shape=[1, event_classes.shape[0]])

<HDF5 dataset "desc": shape (1, 576), type "<u2">

In [181]:
mrk_group.create_dataset('time', data=event_times_in_ms, dtype=np.float64, shape=[event_times_in_ms.shape[0], 1])

<HDF5 dataset "time": shape (576, 1), type "<f8">

In [182]:
assert signal.shape[1] == len(chan_names)
for i_chan in xrange(signal.shape[1]):
    chan_set_name = 'ch' + str(i_chan + 1)
    chan_data = np.ascontiguousarray(deepcopy(signal[:,i_chan]))
    h5file.create_dataset(chan_set_name, data=chan_data, 
                          shape=[1, len(chan_data)], dtype=np.float64)

In [183]:
h5file.close()

## reading in

In [160]:
h5file_read = h5py.File('data/bci-competition-iv/2a-combined-bbci/A03TE.bbci.mat', 'r')

In [ ]:
h5file_read['nfo']['fs'][0,0]

In [62]:
int(h5file_read['nfo']['T'][0,0])

1309305

In [83]:
clab_set

array(['EEG-Fz', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG-C3', 'EEG',
       'EEG-Cz', 'EEG', 'EEG-C4', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG',
       'EEG', 'EEG-Pz', 'EEG', 'EEG'], 
      dtype='|S6')

In [54]:
clab_set = h5file_old['dat']['clab'][:,0]
try:
    all_sensor_names = [''.join(chr(c) for c in h5file_old[obj_ref]) for obj_ref in clab_set]
except KeyError:
    all_sensor_names = clab_set

In [87]:
all_sensor_names

array(['EEG-Fz', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG-C3', 'EEG',
       'EEG-Cz', 'EEG', 'EEG-C4', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG',
       'EEG', 'EEG-Pz', 'EEG', 'EEG'], 
      dtype='|S6')

In [81]:
clab_set

array(['EEG-Fz', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG-C3', 'EEG',
       'EEG-Cz', 'EEG', 'EEG-C4', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG', 'EEG',
       'EEG', 'EEG-Pz', 'EEG', 'EEG'], 
      dtype='|S6')